# Progetto Machine Learning: Riconoscimento di Specie di Uccelli con CNN

Questo notebook implementa un sistema di riconoscimento di specie di uccelli attraverso l'analisi di registrazioni audio della competizione BirdClef 2025. Il progetto utilizza un'architettura CNN per classificare gli audio convertiti in spettrogrammi Mel e include anche un sistema di configurazione automatica dell'ambiente per eseguire il codice su Kaggle, Google Colab o in locale.

## 1. Importazione delle Librerie Necessarie

Importiamo tutte le librerie necessarie per l'elaborazione audio, deep learning e visualizzazione.

In [ ]:
# Librerie di sistema e utilità
import os
import sys
import platform
import time
import warnings
import logging
import datetime
from pathlib import Path
import pprint as pp
import seaborn as sns
from collections import Counter
import IPython.display as ipd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
# Sostituisci le importazioni di Transformers con timm
import timm

# Librerie per data science e manipolazione dati
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split

# Librerie per elaborazione audio
import librosa
import librosa.display

# PyTorch
import torch
import torch.nn.functional as F
from scipy import signal
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchaudio
import torchaudio.transforms as T

# Visualizzazione
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# Ignoriamo i warning
warnings.filterwarnings("ignore")

# Configurazione del logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger('BirdClef')

print("Librerie importate con successo!")
print(f"PyTorch versione: {torch.__version__}")
print(f"timm versione: {timm.__version__}")
print(f"Python versione: {platform.python_version()}")
print(f"Sistema operativo: {platform.system()} {platform.release()}")

In [ ]:
import shutil
import os

# Imposta questo a True per abilitare la cancellazione
clear_working_dir = True

working_dir = '/kaggle/working/'

if clear_working_dir:
    for filename in os.listdir(working_dir):
        file_path = os.path.join(working_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # elimina file o link
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # elimina directory
        except Exception as e:
            print(f'Errore durante la rimozione di {file_path}: {e}')
    print(f"Tutti i file in {working_dir} sono stati rimossi.")
else:
    print("Pulizia disabilitata (clear_working_dir = False)")


## 2. Configurazione dell'Ambiente di Esecuzione

In questa sezione configuriamo l'ambiente di esecuzione in modo che il notebook funzioni sia su Kaggle, che su Google Colab, che in locale. Il codice rileverà automaticamente l'ambiente e configurerà i percorsi di conseguenza.

In [ ]:
# Variabile per impostare manualmente l'ambiente
# Modifica questa variabile in base all'ambiente in uso:
# - 'kaggle' per l'ambiente Kaggle
# - 'colab' per Google Colab
# - 'local' per l'esecuzione in locale
MANUAL_ENVIRONMENT = ''  # Impostare su 'kaggle', 'colab', o 'local' per forzare l'ambiente

def detect_environment():
    """
    Rileva se il notebook è in esecuzione su Kaggle, Google Colab o in locale.
    Rispetta l'impostazione manuale se fornita.
    
    Returns:
        str: 'kaggle', 'colab', o 'local'
    """
    # Se l'ambiente è stato impostato manualmente, usa quello
    if MANUAL_ENVIRONMENT in ['kaggle', 'colab', 'local']:
        print(f"Utilizzo ambiente impostato manualmente: {MANUAL_ENVIRONMENT}")
        return MANUAL_ENVIRONMENT
    
    # Verifica Kaggle con metodo più affidabile
    # Verifica l'esistenza di directory specifiche di Kaggle
    if os.path.exists('/kaggle/working') and os.path.exists('/kaggle/input'):
        print("Rilevato ambiente Kaggle")
        return 'kaggle'
    
    # Verifica se è Google Colab
    try:
        import google.colab
        return 'colab'
    except ImportError:
        pass
    
    # Se non è né Kaggle né Colab, allora è locale
    return 'local'

# Rileva l'ambiente attuale
ENVIRONMENT = detect_environment()
print(f"Ambiente rilevato: {ENVIRONMENT}")

In [ ]:
class Config:
    def __init__(self):
        # Rileva l'ambiente
        self.environment = ENVIRONMENT  # Usa la variabile globale impostata in precedenza
        
        # Imposta i percorsi di base in base all'ambiente
        if self.environment == 'kaggle':
            self.COMPETITION_NAME = "birdclef-2025"
            self.BASE_DIR = f"/kaggle/input/{self.COMPETITION_NAME}"
            self.OUTPUT_DIR = "/kaggle/working"
            self.MODELS_DIR = "/kaggle/input"  # Per i modelli pre-addestrati
            
            # Imposta subito i percorsi derivati per l'ambiente Kaggle
            self._setup_derived_paths()
            
        elif self.environment == 'colab':
            # In Colab, inizializza directory base temporanee
            self.COMPETITION_NAME = "birdclef-2025"
            self.OUTPUT_DIR = "/content/output"
            self.MODELS_DIR = "/content/models"
            
            # Crea le directory di output
            os.makedirs(self.OUTPUT_DIR, exist_ok=True)
            os.makedirs(self.MODELS_DIR, exist_ok=True)
            
            # In Colab, BASE_DIR verrà impostato dopo il download
            # quindi non impostiamo ancora i percorsi derivati
            self.BASE_DIR = "/content/placeholder"  # Verrà sovrascritto dopo il download
            
            # Inizializza i percorsi dei file a None per ora
            self.TRAIN_AUDIO_DIR = None
            self.TEST_SOUNDSCAPES_DIR = None
            self.TRAIN_CSV_PATH = None
            self.TAXONOMY_CSV_PATH = None
            self.SAMPLE_SUB_PATH = None
            
        else:  # locale
            # In ambiente locale, i percorsi dipenderanno dalla tua configurazione
            self.BASE_DIR = os.path.abspath(".")
            self.OUTPUT_DIR = os.path.join(self.BASE_DIR, "output")
            self.MODELS_DIR = os.path.join(self.BASE_DIR, "models")
            
            # Crea le directory se non esistono
            os.makedirs(self.OUTPUT_DIR, exist_ok=True)
            os.makedirs(self.MODELS_DIR, exist_ok=True)
            
            # Imposta i percorsi derivati
            self._setup_derived_paths()
        
        # Parametri per il preprocessing audio - già allineati con vincitori
        self.SR = 32000      # Sample rate
        self.DURATION = 5    # Durata dei clip in secondi
        self.N_MELS = 128    # Numero di bande Mel
        self.N_FFT = 1024    # Dimensione finestra FFT
        self.HOP_LENGTH = 256  # Hop length per STFT
        self.FMIN = 48       # Frequenza minima per lo spettrogramma Mel
        self.FMAX = 15000    # Frequenza massima
        self.POWER = 2.0       # Esponente per calcolo spettrogramma

        self.WIN_LENGTH = None  # Usa n_fft come default
        self.PAD_MODE = "constant"  # Padding mode per spettrogrammi
        self.MEL_SCALE = "htk"      # Scale Mel (Bird25 usa HTK)
        self.NORM = "slaney" 
            
        # Parametri per il training - aggiornati secondo i vincitori
        self.BATCH_SIZE = 64  # Aumentato da 32 a 64 
        self.EPOCHS = 23     # Numero di epoche per il training
        self.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
        self.NUM_WORKERS = 4  # Aumentato per migliorare il data loading

        # Parametri per inference/submission
        self.TEST_CLIP_DURATION = 5  # Durata dei segmenti per la predizione (secondi)
        self.N_CLASSES = 0  # Sarà impostato dopo aver caricato i dati

    def _setup_derived_paths(self):
        """Imposta i percorsi derivati basati su BASE_DIR"""
        # Utilizza la normale divisione di percorso di OS (non il backslash hardcoded)
        self.TRAIN_AUDIO_DIR = os.path.join(self.BASE_DIR, "train_audio")
        self.TEST_SOUNDSCAPES_DIR = os.path.join(self.BASE_DIR, "test_soundscapes")
        self.TRAIN_CSV_PATH = os.path.join(self.BASE_DIR, "train.csv")
        self.TAXONOMY_CSV_PATH = os.path.join(self.BASE_DIR, "taxonomy.csv") 
        self.SAMPLE_SUB_PATH = os.path.join(self.BASE_DIR, "sample_submission.csv")

In [ ]:
config = Config()

# Gestione download dati in Colab con kagglehub
if config.environment == 'colab':
    # Percorsi nella cache di kagglehub
    cache_competition_path = "/root/.cache/kagglehub/competitions/birdclef-2025"
    cache_model_path = "/root/.cache/kagglehub/models/maurocarlu/simplecnn/PyTorch/default/1"
    cache_model_file = os.path.join(cache_model_path, "baseline_bird_cnn_model_val.pth")
    
    # Verifica se i dati sono già presenti nella cache
    data_exists = os.path.exists(os.path.join(cache_competition_path, "train.csv"))
    model_exists = os.path.exists(cache_model_file)
    
    if data_exists and model_exists:
        print("I dati e il modello sono già presenti nella cache. Utilizzo copie esistenti.")
        birdclef_path = cache_competition_path
        model_path = cache_model_path
    else:
        print("Scaricamento dati con kagglehub...")
        
        try:
            import kagglehub
            
            # Scarica solo i dati della competizione se necessario
            if not data_exists:
                print("Download dataset...")
                kagglehub.login()  # Mostra dialog di login interattivo
                birdclef_path = kagglehub.competition_download('birdclef-2025')
            else:
                print("Dataset già presente nella cache.")
                birdclef_path = cache_competition_path
                
            # Scarica solo il modello se necessario
            if not model_exists:
                print("Download modello...")
                kagglehub.login()  # Potrebbe essere necessario riautenticarsi
                model_path = kagglehub.model_download('maurocarlu/simplecnn/PyTorch/default/1')
            else:
                print("Modello già presente nella cache.")
                model_path = cache_model_path
                
            print(f"Download completato.")
            
        except Exception as e:
            print(f"Errore durante il download dei dati: {e}")
            print("Prova ad usare Google Drive o esegui su Kaggle.")
            
            # Se il download fallisce ma i dati esistono parzialmente, usa quelli
            if os.path.exists(cache_competition_path):
                birdclef_path = cache_competition_path
                print(f"Usando i dati esistenti in: {birdclef_path}")
            if os.path.exists(cache_model_path):
                model_path = cache_model_path
                print(f"Usando il modello esistente in: {model_path}")
    
    # Aggiorna i percorsi nella configurazione
    config.BASE_DIR = birdclef_path
    config._setup_derived_paths()
    config.MODELS_DIR = model_path
    model_file = os.path.join(model_path, "baseline_bird_cnn_model_val.pth")
    
    print(f"Dati disponibili in: {config.BASE_DIR}")
    print(f"Modello disponibile in: {model_file}")

# Stampa percorsi aggiornati
print(f"\nPercorso file CSV di training: {config.TRAIN_CSV_PATH}")
print(f"Percorso directory audio di training: {config.TRAIN_AUDIO_DIR}")

### Normalizzazione

In [ ]:
# Crea una singola istanza della trasformazione MelSpectrogram da riutilizzare
mel_transform = T.MelSpectrogram(
    sample_rate=config.SR,
    n_fft=config.N_FFT,
    win_length=None,
    hop_length=config.HOP_LENGTH,
    f_min=config.FMIN,
    f_max=config.FMAX,
    n_mels=config.N_MELS,
    window_fn=torch.hann_window,
    power=config.POWER,
    normalized=False,
    onesided=True,
    norm="slaney",
    mel_scale="htk",
    pad_mode="constant"
)

# Funzione di conversione a dB e normalizzazione
def amplitude_to_db_minmax(spectrogram):
    """
    Converti in dB e applica normalizzazione Min-Max come Bird25
    """
    # Converti in dB
    spectrogram_db = 10.0 * torch.log10(torch.clamp(spectrogram, min=1e-10))
    
    # Normalizzazione Min-Max [0,1] come Bird25
    min_val = torch.min(spectrogram_db)
    max_val = torch.max(spectrogram_db)
    
    # Evita divisione per zero
    range_val = max_val - min_val
    if range_val > 1e-8:
        normalized = (spectrogram_db - min_val) / range_val
    else:
        normalized = torch.zeros_like(spectrogram_db)
    
    return normalized


# Funzione per gestire file audio troppo corti (come Bird25)
def handle_short_audio(audio_data, target_samples):
    """
    Gestisce file audio troppo corti concatenandoli come Bird25
    """
    if len(audio_data) < target_samples:
        # Calcola quante copie servono
        n_copy = math.ceil(target_samples / len(audio_data))
        if n_copy > 1:
            # Concatena l'audio n_copy volte
            audio_data = np.concatenate([audio_data] * n_copy)
    
    return audio_data[:target_samples]  # Tronca alla lunghezza esatta

## 3. Configurazione del Modello e Parametri

Definiamo i parametri di configurazione per il preprocessamento audio, la creazione dello spettrogramma Mel e l'addestramento della CNN.

In [ ]:
# I parametri principali sono già definiti nella classe Config
# Verifichiamo l'esistenza delle directory e creiamo quelle necessarie per l'output

def setup_output_directories():
    """
    Configura le directory per l'output del progetto.
    
    Returns:
        dict: Dictionary con i percorsi delle directory di output
    """
    # Directory principale di output
    output_dir = config.OUTPUT_DIR
    
    # Sotto-directory per diversi tipi di output
    dirs = {
        'checkpoints': os.path.join(output_dir, 'checkpoints'),
        'tensorboard': os.path.join(output_dir, 'tensorboard_logs'),
        'predictions': os.path.join(output_dir, 'predictions'),
        'submissions': os.path.join(output_dir, 'submissions'),
        'visualizations': os.path.join(output_dir, 'visualizations'),
    }
    
    # Crea tutte le directory
    for dir_name, dir_path in dirs.items():
        os.makedirs(dir_path, exist_ok=True)
        print(f"Directory '{dir_name}' creata/verificata in: {dir_path}")
    
    return dirs

# Configura le directory di output
output_dirs = setup_output_directories()

# Crea un file di log per tenere traccia dei risultati
log_file_path = os.path.join(config.OUTPUT_DIR, f"experiment_log_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")

with open(log_file_path, 'w') as log_file:
    log_file.write(f"=== BirdClef Experiment Log ===\n")
    log_file.write(f"Date: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    log_file.write(f"Environment: {config.environment}\n\n")
    log_file.write("Output directories:\n")
    for dir_name, dir_path in output_dirs.items():
        log_file.write(f"- {dir_name}: {dir_path}\n")

print(f"File di log creato in: {log_file_path}")

# Memorizziamo i parametri di configurazione principali per l'addestramento
print("\nParametri di configurazione principali:")
print(f"- Sample rate: {config.SR} Hz")
print(f"- Durata clip audio: {config.DURATION} secondi")
print(f"- Numero bande Mel: {config.N_MELS}")
print(f"- Dimensione FFT: {config.N_FFT}")
print(f"- Hop length: {config.HOP_LENGTH}")
print(f"- Device: {config.DEVICE}")
print(f"- Batch size: {config.BATCH_SIZE}")
print(f"- Epoche: {config.EPOCHS}")

## 4. Caricamento e Preprocessing dei Dati

In questa sezione carichiamo i metadati dal file CSV di training, creiamo codifiche one-hot per le etichette delle specie e implementiamo funzioni per il caricamento e preprocessamento dei file audio.

In [ ]:
# Caricamento dei metadati
def load_metadata():
    """
    Carica e prepara i metadati dal file CSV di training.
    
    Returns:
        tuple: training_df, all_species, labels_one_hot
    """
    print(f"Caricamento metadati da: {config.TRAIN_CSV_PATH}")
    train_df = pd.read_csv(config.TRAIN_CSV_PATH)
    sample_sub_df = pd.read_csv(config.SAMPLE_SUB_PATH)
    
    # Estrai tutte le etichette uniche
    train_primary_labels = train_df['primary_label'].unique()
    train_secondary_labels = set([lbl for sublist in train_df['secondary_labels'].apply(eval) 
                                 for lbl in sublist if lbl])
    submission_species = sample_sub_df.columns[1:].tolist()  # Escludi row_id
    
    # Combina tutte le possibili etichette
    all_species = sorted(list(set(train_primary_labels) | train_secondary_labels | set(submission_species)))
    N_CLASSES = len(all_species)
    config.N_CLASSES = N_CLASSES  # Aggiorna il numero di classi nella configurazione
    
    print(f"Numero totale di specie trovate: {N_CLASSES}")
    print(f"Prime 10 specie: {all_species[:10]}")
    
    # Crea mappatura etichette-indici
    species_to_int = {species: i for i, species in enumerate(all_species)}
    int_to_species = {i: species for species, i in species_to_int.items()}
    
    # Aggiungi indici numerici al dataframe
    train_df['primary_label_int'] = train_df['primary_label'].map(species_to_int)
    
    # Prepara target multi-etichetta
    mlb = MultiLabelBinarizer(classes=all_species)
    mlb.fit(None)  # Fit con tutte le classi
    
    def get_multilabel(row):
        labels = eval(row['secondary_labels'])  # Valuta la lista di stringhe in modo sicuro
        labels.append(row['primary_label'])
        return list(set(labels))  # Assicura etichette uniche
    
    train_df['all_labels'] = train_df.apply(get_multilabel, axis=1)
    train_labels_one_hot = mlb.transform(train_df['all_labels'])
    
    print(f"Forma delle etichette one-hot: {train_labels_one_hot.shape}")
    
    return train_df, all_species, train_labels_one_hot, species_to_int, int_to_species

# Carica i metadati
train_df, all_species, train_labels_one_hot, species_to_int, int_to_species = load_metadata()

# Suddividi i dati in training e validation
def split_data(train_df, labels_one_hot, test_size=0.2, random_state=42):
    """
    Suddivide il dataset in set di training e validation.
    
    Args:
        train_df: DataFrame con i metadati
        labels_one_hot: Array di etichette one-hot
        test_size: Percentuale dei dati da usare per validation
        random_state: Seed per riproducibilità
        
    Returns:
        tuple: X_train_df, X_val_df, y_train_one_hot, y_val_one_hot
    """
    # Indici per lo split
    train_indices, val_indices = train_test_split(
        range(len(train_df)),
        test_size=test_size,
        random_state=random_state
    )
    
    # Crea i dataframe e gli array di etichette splittati
    X_train_df = train_df.iloc[train_indices].reset_index(drop=True)
    X_val_df = train_df.iloc[val_indices].reset_index(drop=True)
    
    y_train_one_hot = labels_one_hot[train_indices]
    y_val_one_hot = labels_one_hot[val_indices]
    
    print(f"Dimensioni Training Set: {X_train_df.shape}, Etichette: {y_train_one_hot.shape}")
    print(f"Dimensioni Validation Set: {X_val_df.shape}, Etichette: {y_val_one_hot.shape}")
    
    return X_train_df, X_val_df, y_train_one_hot, y_val_one_hot

# Suddividi i dati in training e validation
X_train_df, X_val_df, y_train_one_hot, y_val_one_hot = split_data(train_df, train_labels_one_hot)

    
# Per Kaggle, dovremo creare un dataset speciale per le soundscapes di test
# Questo verrà utilizzato direttamente nella fase di generazione della submission
# Non creiamo X_test_df e test_dataset per ora
X_test_df = None
y_test_one_hot = None

## Funzione di bilanciamento del dataset - Cancella una percentuale di esempi dalle classi molto numerose

In [ ]:
def create_balanced_dataset_df(train_df, labels_one_hot, abundant_class_threshold=200, remove_percentage=0.3, random_state=42):
    """
    Crea un DataFrame bilanciato rimuovendo parte degli esempi con rating bassi dalle classi abbondanti.
    
    Args:
        train_df: DataFrame originale
        labels_one_hot: Array di etichette one-hot
        abundant_class_threshold: Soglia per definire una classe come "abbondante"
        remove_percentage: Percentuale di esempi con rating 1-3 da rimuovere dalle classi abbondanti
        random_state: Seed per riproducibilità
        
    Returns:
        tuple: (DataFrame bilanciato, etichette one-hot bilanciate)
    """
    # Conta esempi per ogni classe
    class_counts = train_df['primary_label'].value_counts()
    
    # Identifica classi abbondanti
    abundant_classes = class_counts[class_counts > abundant_class_threshold].index.tolist()
    print(f"Classi identificate come abbondanti (>{abundant_class_threshold} esempi): {len(abundant_classes)}")
    
    # Copia il DataFrame originale
    balanced_df = train_df.copy()
    rows_to_drop = []
    
    # Contatori per statistiche
    total_removed = 0
    removed_by_class = {}
    
    # Per ogni classe abbondante
    for cls in abundant_classes:
        # Filtra esempi con rating 1-3 per questa classe
        low_quality_mask = (balanced_df['primary_label'] == cls) & (balanced_df['rating'].isin([1, 2, 3]))
        low_quality_indices = balanced_df[low_quality_mask].index.tolist()
        
        # Numero di esempi da rimuovere
        n_to_remove = int(len(low_quality_indices) * remove_percentage)
        
        # Seleziona casualmente gli indici da rimuovere
        np.random.seed(random_state)
        if n_to_remove > 0:
            indices_to_remove = np.random.choice(low_quality_indices, size=n_to_remove, replace=False)
            
            # Memorizza gli indici da rimuovere
            rows_to_drop.extend(indices_to_remove)
            
            # Aggiorna statistiche
            removed_by_class[cls] = n_to_remove
            total_removed += n_to_remove
    
    # Rimuovi le righe selezionate
    if rows_to_drop:
        balanced_df = balanced_df.drop(rows_to_drop).reset_index(drop=True)
        
        # Aggiorna anche le etichette one-hot rimuovendo gli stessi indici
        mask = np.ones(len(train_df), dtype=bool)
        mask[rows_to_drop] = False
        balanced_labels = labels_one_hot[mask]
    else:
        balanced_labels = labels_one_hot
    
    # Statistiche finali
    print(f"Totale esempi rimossi: {total_removed} ({total_removed/len(train_df):.1%} del dataset originale)")
    print(f"Dimensione dataset originale: {len(train_df)}")
    print(f"Dimensione dataset bilanciato: {len(balanced_df)}")
    
    # Visualizza le prime 5 classi con maggiori rimozioni
    if removed_by_class:
        top_removed = sorted(removed_by_class.items(), key=lambda x: x[1], reverse=True)[:5]
        print("\nClassi con maggior numero di esempi rimossi:")
        for cls, count in top_removed:
            original = class_counts[cls]
            remaining = original - count
            print(f"- {cls}: {count} rimossi, {remaining}/{original} rimanenti ({remaining/original:.1%})")
    else:
        print("Nessun esempio rimosso.")
    
    return balanced_df, balanced_labels

## 4.5 Analisi Esplorativa dei Dati (EDA)

In questa sezione esploreremo le caratteristiche del dataset per comprendere meglio la distribuzione delle specie, le proprietà audio e identificare eventuali pattern nei dati.

In [ ]:
# Configurazione stile visualizzazioni
plt.style.use('seaborn-whitegrid')
sns.set(style="whitegrid", font_scale=1.1)
plt.rcParams['figure.figsize'] = [12, 6]

print("=== Statistiche di base del dataset ===")
print(f"Numero totale di registrazioni: {len(train_df)}")
print(f"Numero di specie uniche nel dataset: {len(all_species)}")
print(f"Campi disponibili nei metadati: {train_df.columns.tolist()}")

# Verifichiamo i dati mancanti
missing_data = train_df.isnull().sum()
print("\n=== Valori mancanti ===")
print(missing_data[missing_data > 0])

# 1. Distribuzione delle specie nel dataset (visualizzazione migliorata)
print("\n=== Analisi delle Specie ===")
primary_species_count = train_df['primary_label'].value_counts()

# Plot combinato: distribuzione delle specie con evidenza delle classi rare
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# A sinistra: top 20 specie più rappresentate
sns.barplot(x=primary_species_count.head(20).index, y=primary_species_count.head(20).values, ax=ax1)
ax1.set_title('Top 20 Specie per Numero di Registrazioni')
ax1.set_xlabel('Specie')
ax1.set_ylabel('Numero di Registrazioni')
ax1.tick_params(axis='x', rotation=90)

# A destra: distribuzione del numero di esempi per specie
sns.histplot(primary_species_count, bins=30, kde=True, ax=ax2)
ax2.set_title('Distribuzione del Numero di Registrazioni per Specie')
ax2.set_xlabel('Numero di Registrazioni')
ax2.set_ylabel('Conteggio Specie')
ax2.axvline(x=primary_species_count.median(), color='r', linestyle='--', 
            label=f'Mediana: {primary_species_count.median()}')
ax2.axvline(x=primary_species_count.mean(), color='g', linestyle='--', 
            label=f'Media: {primary_species_count.mean():.1f}')
ax2.axvline(x=50, color='orange', linestyle=':', label='Soglia classi rare (50)')
ax2.legend()

plt.tight_layout()
plt.show()

# Calcolo dell'indice di Gini per misurare lo sbilanciamento
def gini_coefficient(x):
    x = np.sort(x)
    n = len(x)
    index = np.arange(1, n+1)
    return (np.sum((2*index - n - 1) * x)) / (n * np.sum(x))

gini = gini_coefficient(primary_species_count.values)
print(f"\nIndice di Gini per la distribuzione delle specie: {gini:.4f}")
print(f"Questo indica {'un alto' if gini > 0.6 else 'un moderato' if gini > 0.3 else 'un basso'} livello di sbilanciamento nel dataset.")

# 2. NUOVA ANALISI: Rating per le classi con pochi esempi
# Definisco la soglia per le classi rare (< 50 esempi)
RARE_CLASS_THRESHOLD = 50
rare_species = primary_species_count[primary_species_count < RARE_CLASS_THRESHOLD].index.tolist()
print(f"\n=== Analisi dei Rating per Classi Rare (<{RARE_CLASS_THRESHOLD} esempi) ===")
print(f"Numero di classi rare: {len(rare_species)} su {len(all_species)} totali ({len(rare_species)/len(all_species):.1%})")

# Raccolgo i dati sui rating per le classi rare
rare_class_ratings = []
for species in rare_species:
    species_df = train_df[train_df['primary_label'] == species]
    ratings = species_df['rating'].fillna(0).tolist()  # Sostituisco NaN con 0 (nessun rating)
    
    # Statistiche per questa specie
    rare_class_ratings.append({
        'species': species,
        'count': len(species_df),
        'avg_rating': np.mean(ratings),
        'ratings': ratings,
        'rating_counts': {r: ratings.count(r) for r in set(ratings)}
    })

# Creo DataFrame per analisi
rare_ratings_df = pd.DataFrame(rare_class_ratings)
rare_ratings_df = rare_ratings_df.sort_values('count')

# Visualizzazione: Rating medi vs Conteggio per le classi rare
plt.figure(figsize=(12, 6))

# Heatmap: distribuzione dei rating per classi rare
n_rare_to_show = min(30, len(rare_ratings_df))  # Mostra max 30 classi per leggibilità
rare_sample = rare_ratings_df.head(n_rare_to_show)

# Preparo i dati per la heatmap
heatmap_data = []
rating_values = [0, 1, 2, 3, 4, 5]  # Tutti i possibili rating
for _, row in rare_sample.iterrows():
    species_data = [row['species'], row['count']]
    for rating in rating_values:
        species_data.append(row['rating_counts'].get(rating, 0))
    heatmap_data.append(species_data)

# Creo DataFrame per la heatmap
heatmap_df = pd.DataFrame(
    heatmap_data, 
    columns=['species', 'count'] + [f'rating_{r}' for r in rating_values]
)

# Plot combinato con scatter plot e heatmap
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))

# Scatter plot: rating medio vs numero esempi
sns.scatterplot(
    x='count', 
    y='avg_rating', 
    data=rare_ratings_df, 
    ax=ax1, 
    alpha=0.7,
    hue='count',
    palette='viridis',
    size='count',
    sizes=(20, 200)
)
ax1.set_title('Rating Medio vs Numero di Esempi per Classi Rare')
ax1.set_xlabel('Numero di Esempi')
ax1.set_ylabel('Rating Medio')
ax1.grid(True)

# Mostra statistiche
avg_rating_rare = rare_ratings_df['avg_rating'].mean()
ax1.axhline(y=avg_rating_rare, color='r', linestyle='--', 
           label=f'Rating medio classi rare: {avg_rating_rare:.2f}')
ax1.legend()

# Heatmap: distribuzione dei rating per le classi più rare
pivot_data = pd.DataFrame({
    'species': heatmap_df['species'],
    'Rating 0': heatmap_df['rating_0'],
    'Rating 1': heatmap_df['rating_1'],
    'Rating 2': heatmap_df['rating_2'],
    'Rating 3': heatmap_df['rating_3'],
    'Rating 4': heatmap_df['rating_4'],
    'Rating 5': heatmap_df['rating_5'],
}).set_index('species')

sns.heatmap(pivot_data, cmap="YlGnBu", annot=True, fmt='g', ax=ax2)
ax2.set_title(f'Distribuzione dei Rating nelle {n_rare_to_show} Classi più Rare')

plt.tight_layout()
plt.show()

# Statistiche aggregate sui rating per le classi rare
print("\nStatistiche sui rating per le classi rare:")
print(f"- Rating medio complessivo: {rare_ratings_df['avg_rating'].mean():.2f}")
print(f"- Percentuale di registrazioni senza rating (0): {sum(r['rating_counts'].get(0, 0) for r in rare_class_ratings) / sum(r['count'] for r in rare_class_ratings):.1%}")

# Analisi delle classi estremamente rare (≤ 5 esempi)
very_rare_species = primary_species_count[primary_species_count <= 5].index.tolist()
print(f"\nClassi estremamente rare (≤ 5 esempi): {len(very_rare_species)}")

very_rare_df = train_df[train_df['primary_label'].isin(very_rare_species)]
print("Dettaglio delle registrazioni per le classi estremamente rare:")
for species in very_rare_species:
    species_data = train_df[train_df['primary_label'] == species]
    ratings = species_data['rating'].fillna(0).tolist()
    print(f"- {species}: {len(species_data)} esempi, ratings: {ratings}")


# Identificazione delle classi con solo rating molto bassi (1-2)
print("\n=== Analisi delle Classi con SOLO Rating Molto Bassi (1-2) ===")

# Funzione per verificare se una specie ha esclusivamente rating tra 1-2
def has_only_low_ratings(species_ratings):
    valid_ratings = [r for r in species_ratings if pd.notna(r) and r != 0]  # Escludi NaN e rating=0
    if not valid_ratings:  # Se non ci sono rating validi
        return False
    return all(1 <= r <= 2 for r in valid_ratings)  # Modificato: ora solo 1-2

# Raggruppa per specie e analizza
low_rating_species = []
for species, group in train_df.groupby('primary_label'):
    ratings = group['rating'].tolist()
    if has_only_low_ratings(ratings):
        low_rating_species.append({
            'species': species,
            'count': len(ratings),
            'avg_rating': np.nanmean([r for r in ratings if pd.notna(r) and r != 0]),
            'ratings': sorted([r for r in ratings if pd.notna(r) and r != 0])
        })

# Crea DataFrame e visualizza risultati
if low_rating_species:
    low_rating_df = pd.DataFrame(low_rating_species).sort_values('count', ascending=False)
    
    print(f"Trovate {len(low_rating_species)} classi con SOLO rating molto bassi (1-2):")
    
    # Visualizzazione
    plt.figure(figsize=(12, 6))
    plt.bar(low_rating_df['species'], low_rating_df['avg_rating'], alpha=0.7, color='tomato')
    plt.axhline(y=1.5, color='r', linestyle='--', label='Media teorica = 1.5')
    plt.title('Classi con Esclusivamente Rating Molto Bassi (1-2)')
    plt.xlabel('Specie')
    plt.ylabel('Rating Medio')
    plt.xticks(rotation=90)
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # Mostra dettagli delle prime 10 classi
    print("\nDettagli delle prime 10 classi con solo rating molto bassi:")
    for i, row in low_rating_df.head(10).iterrows():
        print(f"- {row['species']}: {row['count']} clip, rating medio: {row['avg_rating']:.2f}, ratings: {row['ratings']}")
    
    # Cerca sovrapposizione con classi rare
    overlap = [s for s in low_rating_df['species'] if s in rare_species]
    print(f"\nSovrapposizione con classi rare (<{RARE_CLASS_THRESHOLD} esempi): {len(overlap)} classi")
    if overlap:
        print(f"Le classi rare che hanno solo rating molto bassi: {overlap[:10]}{'...' if len(overlap) > 10 else ''}")
else:
    print("Nessuna classe ha esclusivamente rating da 1 a 2.")

print("\n=== Conclusioni dall'Analisi delle Classi Rare ===")
print(f"1. Abbiamo {len(rare_species)} classi rare (<{RARE_CLASS_THRESHOLD} esempi)")
print(f"2. Di queste, {len(very_rare_species)} hanno 5 o meno esempi")
print("3. La qualità delle registrazioni (rating) è un fattore critico per le classi rare")
print("4. Le classi estremamente rare richiedono tecniche speciali (data augmentation, few-shot learning)")

## Data Augmentation: Implementazione delle Tecniche dei Vincitori

In questa sezione implementiamo le tre tecniche di data augmentation che hanno contribuito significativamente alle performance dei vincitori:
1. **Random Segment Selection** - Estrae segmenti casuali dalle registrazioni audio
2. **XY Masking** - Applica maschere casuali sugli assi tempo e frequenza degli spettrogrammi Mel
3. **Horizontal CutMix** - Combina parti di spettrogrammi da diverse registrazioni

La classe `AudioAugmentations` gestisce tutte queste trasformazioni in modo unificato.

In [ ]:
class SEDAugmentations:
    """Augmentations specifiche per SED utilizzate dai vincitori"""
    
    def __init__(self, 
                 p_mixup=0.5,
                 p_background_noise=0.3,
                 p_random_filtering=0.4,
                 p_spec_freq=0.3,
                 p_spec_time=0.3):
        """
        Args:
            p_mixup: Probabilità di applicare Mixup
            p_background_noise: Probabilità di aggiungere rumore di background
            p_random_filtering: Probabilità di applicare random filtering
            p_spec_freq: Probabilità di frequency masking
            p_spec_time: Probabilità di time masking
        """
        self.p_mixup = p_mixup
        self.p_background_noise = p_background_noise
        self.p_random_filtering = p_random_filtering
        self.p_spec_freq = p_spec_freq
        self.p_spec_time = p_spec_time
        
        # Parametri per SpecAug (come nei tuoi esempi)
        self.freq_max_length = 10
        self.freq_max_lines = 3
        self.time_max_length = 20
        self.time_max_lines = 3
    
    def apply_mixup_audio(self, audio1, audio2, alpha=0.2):
        """Applica Mixup direttamente sull'audio"""
        if np.random.random() > self.p_mixup:
            return audio1, 1.0  # Nessun mixup, peso = 1.0
        
        # Genera lambda dalla distribuzione Beta
        lam = np.random.beta(alpha, alpha) if alpha > 0 else 0.5
        
        # Assicurati che entrambi gli audio abbiano la stessa lunghezza
        min_len = min(len(audio1), len(audio2))
        audio1 = audio1[:min_len]
        audio2 = audio2[:min_len]
        
        # Mix degli audio
        mixed_audio = lam * audio1 + (1 - lam) * audio2
        
        return mixed_audio, lam  # Ritorna anche il peso per le etichette
    
    def add_background_noise(self, audio, noise_factor=0.005):
        """Aggiunge rumore di background (simula Zenodo nocall)"""
        if np.random.random() > self.p_background_noise:
            return audio
        
        # Genera rumore gaussiano
        noise = np.random.normal(0, noise_factor, len(audio))
        
        # Simula anche rumore ambientale a bassa frequenza
        if np.random.random() < 0.5:
            # Rumore a bassa frequenza (vento, etc.)
            t = np.linspace(0, len(audio) / config.SR, len(audio))
            low_freq_noise = noise_factor * 0.5 * np.sin(2 * np.pi * np.random.uniform(1, 10) * t)
            noise += low_freq_noise
        
        return audio + noise
    
    def apply_random_filtering(self, audio, sr=config.SR):
        """Random Filtering - equalizzatore casuale semplificato"""
        if np.random.random() > self.p_random_filtering:
            return audio
        
        # Scegli tipo di filtro casualmente
        filter_type = np.random.choice(['highpass', 'lowpass', 'bandpass'])
        
        if filter_type == 'highpass':
            # High-pass filter (rimuove basse frequenze)
            cutoff = np.random.uniform(100, 1000)  # Hz
            sos = signal.butter(4, cutoff, btype='highpass', fs=sr, output='sos')
            
        elif filter_type == 'lowpass':
            # Low-pass filter (rimuove alte frequenze)
            cutoff = np.random.uniform(8000, 15000)  # Hz
            sos = signal.butter(4, cutoff, btype='lowpass', fs=sr, output='sos')
            
        else:  # bandpass
            # Band-pass filter
            low = np.random.uniform(200, 1000)
            high = np.random.uniform(8000, 14000)
            sos = signal.butter(4, [low, high], btype='bandpass', fs=sr, output='sos')
        
        try:
            filtered_audio = signal.sosfilt(sos, audio)
            return filtered_audio.astype(np.float32)
        except:
            return audio  # Se il filtro fallisce, ritorna audio originale
    
    def apply_spec_augment(self, spectrogram):
        """Applica SpecAugment con i parametri che hai specificato"""
        spec = spectrogram.clone()
        
        if len(spec.shape) == 3:  # [C, H, W]
            C, H, W = spec.shape
        else:
            raise ValueError(f"Forma spettrogramma non supportata: {spec.shape}")
        
        # Frequency Masking
        if np.random.random() < self.p_spec_freq:
            for _ in range(np.random.randint(1, self.freq_max_lines + 1)):
                mask_height = np.random.randint(1, min(self.freq_max_length + 1, H // 4))
                mask_start = np.random.randint(0, max(1, H - mask_height))
                spec[:, mask_start:mask_start + mask_height, :] = 0
        
        # Time Masking
        if np.random.random() < self.p_spec_time:
            for _ in range(np.random.randint(1, self.time_max_lines + 1)):
                mask_width = np.random.randint(1, min(self.time_max_length + 1, W // 4))
                mask_start = np.random.randint(0, max(1, W - mask_width))
                spec[:, :, mask_start:mask_start + mask_width] = 0
        
        return spec
    
    def apply_all_audio_augs(self, audio, other_audio=None, sr=config.SR):
        """Applica tutte le augmentations audio in sequenza"""
        # 1. Mixup (se disponibile altro audio)
        mix_weight = 1.0
        if other_audio is not None:
            audio, mix_weight = self.apply_mixup_audio(audio, other_audio)
        
        # 2. Background Noise
        audio = self.add_background_noise(audio)
        
        # 3. Random Filtering
        audio = self.apply_random_filtering(audio, sr)
        
        return audio, mix_weight

## 5. Dataset PyTorch per Dati Audio

## Dataset PyTorch con Supporto Integrato per Data Augmentation

Implementiamo due classi di dataset:
- `RandomSegmentBirdDataset` 
  - Estrae segmenti randomici di 15 secondi in base alla durata delle registrazioni
  - Applica le tecniche di data augmentation durante il caricamento

### RandomSegmentBirdDataset

In [ ]:
class RandomSegmentBirdDatasetSED(Dataset):
    """Dataset SED che passa audio crudo al modello"""
    def __init__(self, df, audio_dir, labels_one_hot, 
                 sed_augmentations=None, spec_augmentations=None):
        self.df = df
        self.audio_dir = audio_dir
        self.labels = labels_one_hot
        self.sed_augmentations = sed_augmentations
        # Nota: spec_augmentations non serve qui, viene gestito nel modello
        
        print("✅ Dataset SED con audio crudo inizializzato...")
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        filename = row['filename']
        file_path = os.path.join(self.audio_dir, filename)
        
        if not os.path.exists(file_path):
            print(f"⚠️ File non trovato: {file_path}")
            # Ritorna audio crudo dummy, non spettrogramma
            dummy_audio = torch.zeros(config.SR * config.DURATION, dtype=torch.float32)
            dummy_label = torch.zeros(config.N_CLASSES, dtype=torch.float32)
            return dummy_audio, dummy_label
        
        try:
            # Carica audio con torchaudio
            waveform, sr = torchaudio.load(file_path)
            
            # Converti a mono se necessario
            if waveform.shape[0] > 1:
                waveform = waveform[0:1]
            
            # Ricampiona se necessario
            if sr != config.SR:
                resampler = T.Resample(sr, config.SR)
                waveform = resampler(waveform)
            
            # Converti in numpy per augmentations
            audio_data = waveform.squeeze().numpy()
            target_samples = int(config.SR * config.DURATION)
            
            # Gestione file corti
            if len(audio_data) < target_samples:
                audio_data = handle_short_audio(audio_data, target_samples)
            
            # Estrazione randomica
            if len(audio_data) > target_samples:
                max_start_idx = len(audio_data) - target_samples
                start_idx = np.random.randint(0, max_start_idx)
                audio_data = audio_data[start_idx:start_idx + target_samples]
            else:
                audio_data = audio_data[:target_samples]
            
            # Applica augmentations SED sull'audio crudo
            mix_weight = 1.0
            if self.sed_augmentations is not None and len(self.df) > 1:
                # Mixup con altro audio (stesso codice di prima)
                other_idx = np.random.randint(0, len(self.df))
                if other_idx != idx:
                    other_row = self.df.iloc[other_idx]
                    other_file_path = os.path.join(self.audio_dir, other_row['filename'])
                    
                    if os.path.exists(other_file_path):
                        try:
                            other_waveform, other_sr = torchaudio.load(other_file_path)
                            if other_waveform.shape[0] > 1:
                                other_waveform = other_waveform[0:1]
                            if other_sr != config.SR:
                                other_resampler = T.Resample(other_sr, config.SR)
                                other_waveform = other_resampler(other_waveform)
                            
                            other_audio_data = other_waveform.squeeze().numpy()
                            if len(other_audio_data) < target_samples:
                                other_audio_data = handle_short_audio(other_audio_data, target_samples)
                            
                            if len(other_audio_data) > target_samples:
                                other_start_idx = np.random.randint(0, len(other_audio_data) - target_samples)
                                other_audio_data = other_audio_data[other_start_idx:other_start_idx + target_samples]
                            else:
                                other_audio_data = other_audio_data[:target_samples]
                            
                            audio_data, mix_weight = self.sed_augmentations.apply_all_audio_augs(
                                audio_data, other_audio_data
                            )
                        except:
                            audio_data, mix_weight = self.sed_augmentations.apply_all_audio_augs(audio_data)
                    else:
                        audio_data, mix_weight = self.sed_augmentations.apply_all_audio_augs(audio_data)
                else:
                    audio_data, mix_weight = self.sed_augmentations.apply_all_audio_augs(audio_data)
            
            # CRUCIALE: Ritorna audio crudo 1D, non spettrogramma
            audio_tensor = torch.tensor(audio_data, dtype=torch.float32)
            
        except Exception as e:
            print(f"❌ Errore nel caricamento di {file_path}: {e}")
            audio_tensor = torch.zeros(config.SR * config.DURATION, dtype=torch.float32)
            mix_weight = 1.0
        
        # Gestione etichette con mixup
        label_tensor = torch.tensor(self.labels[idx], dtype=torch.float32)
        
        if mix_weight < 1.0 and 'other_idx' in locals() and other_idx != idx:
            try:
                other_label_tensor = torch.tensor(self.labels[other_idx], dtype=torch.float32)
                label_tensor = mix_weight * label_tensor + (1 - mix_weight) * other_label_tensor
            except:
                pass
        
        return audio_tensor, label_tensor

### Creazione dataset e dataloader con RandomSegmentBirdDataset

### Creazione dei DataLoader con Augmentation

In questa sezione:
1. Applichiamo il bilanciamento strategico al dataset di training
2. Creiamo l'istanza di AudioAugmentations con le probabilità ottimali
3. Configuriamo i dataloader con le funzioni di collate personalizzate
4. Analizziamo la distribuzione dei segmenti nel dataset risultante

In [ ]:
# Applica bilanciamento
# Applica bilanciamento
print("\n🔄 Bilanciamento del dataset di training...")
X_train_df_balanced, y_train_one_hot_balanced = create_balanced_dataset_df(
    X_train_df, 
    y_train_one_hot,
    abundant_class_threshold=100,
    remove_percentage=0.5
)

# ⚠️ CREA LE NUOVE AUGMENTATIONS SED DEI VINCITORI
print("🔄 Inizializzazione augmentations SED dei vincitori...")

sed_augmentations = SEDAugmentations(
    p_mixup=0.6,           # Come specificato dagli utenti
    p_background_noise=0.4,
    p_random_filtering=0.5,
    p_spec_freq=0.4,       # Come specificato
    p_spec_time=0.4        # Come specificato
)

# SpecAugment separato per il modello
spec_augmentations = SEDAugmentations(
    p_mixup=0.0,           # Già fatto nel dataset
    p_background_noise=0.0,
    p_random_filtering=0.0,
    p_spec_freq=0.3,       # Solo SpecAug nel modello
    p_spec_time=0.3
)

# Validation senza augmentations aggressive
val_sed_augmentations = SEDAugmentations(
    p_mixup=0.0,
    p_background_noise=0.1,  # Solo un po' di rumore
    p_random_filtering=0.0,
    p_spec_freq=0.0,
    p_spec_time=0.0
)

# ⚠️ DATASET SED CON AUGMENTATIONS DEI VINCITORI
print("🔄 Creazione dataset SED con augmentations...")

train_dataset = RandomSegmentBirdDatasetSED(
    X_train_df_balanced, 
    config.TRAIN_AUDIO_DIR, 
    y_train_one_hot_balanced,
    sed_augmentations=sed_augmentations,
    spec_augmentations=spec_augmentations
)

val_dataset = RandomSegmentBirdDatasetSED(
    X_val_df, 
    config.TRAIN_AUDIO_DIR, 
    y_val_one_hot,
    sed_augmentations=val_sed_augmentations,
    spec_augmentations=None  # No SpecAug in validation
)

# DataLoader semplificati per SED
train_loader = DataLoader(
    train_dataset, 
    batch_size=config.BATCH_SIZE,
    shuffle=True,
    num_workers=config.NUM_WORKERS,
    pin_memory=True
    # NO collate_fn - SED gestisce tutto internamente
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=config.BATCH_SIZE, 
    shuffle=False,
    num_workers=config.NUM_WORKERS, 
    pin_memory=True
)

print(f"✅ Training batches per epoca: {len(train_loader)}")
print(f"✅ Validation batches per epoca: {len(val_loader)}")
print("🎯 Dataset SED con augmentations dei vincitori configurato!")

## 6. Definizione del Modello CNN

## Modello EfficientNet con Head Personalizzata

Implementiamo un modello basato su EfficientNet-B0 preaddestrato, con:
- Supporto per input a singolo canale (spettrogrammi Mel)
- Testa di classificazione personalizzata con dropout e normalizzazione batch
- Parametri differenziati per l'ottimizzazione
- Gestione automatica dei checkpoint e dei pesi preaddestrati

In [ ]:
def init_layer(layer):
    """Inizializza i layer come in Bird25"""
    nn.init.xavier_uniform_(layer.weight)
    if hasattr(layer, "bias"):
        if layer.bias is not None:
            layer.bias.data.fill_(0.0)

def init_bn(bn):
    """Inizializza batch normalization come in Bird25"""
    bn.bias.data.fill_(0.0)
    bn.weight.data.fill_(1.0)

class AttBlockV2(nn.Module):
    """Attention Block V2 - IDENTICO a Bird25"""
    def __init__(self, in_features: int, out_features: int, activation="linear"):
        super().__init__()
        self.activation = activation
        
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True,
        )
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True,
        )
        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.tanh(self.att(x)), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == "linear":
            return x
        elif self.activation == "sigmoid":
            return torch.sigmoid(x)

In [ ]:
class WeightedFocalBCELoss(nn.Module):
    def __init__(self, class_counts, gamma=2, alpha_rare=3.0, alpha_common=0.5):
        super().__init__()
        self.gamma = gamma
        
        # MIGLIORAMENTO: Calcolo pesi più aggressivo per sbilanciamento estremo
        total_samples = sum(class_counts.values())
        self.class_weights = {}
        
        for species, count in class_counts.items():
            frequency = count / total_samples
            
            # CORRETTO: Soglie più aggressive per sbilanciamento estremo
            if frequency < 0.002:  # < 0.2% → peso molto alto (classi estremamente rare)
                weight = alpha_rare * 3  # 9.0 invece di 3.0
            elif frequency < 0.005:  # < 0.5% → peso molto alto
                weight = alpha_rare * 2  # 6.0 invece di 3.0
            elif frequency < 0.01:   # < 1% → peso alto  
                weight = alpha_rare      # 3.0
            elif frequency > 0.05:   # > 5% → peso basso
                weight = alpha_common    # 0.5
            elif frequency > 0.1:    # > 10% → peso molto basso
                weight = alpha_common * 0.3  # 0.15 invece di 0.25
            else:
                # Peso inversamente proporzionale con smoothing migliorato
                weight = min(1.0 / (frequency + 0.0005), 4.0)  # Cap più alto per classi rare
        
            self.class_weights[species] = weight
        
        # Converte in tensore per GPU con ordine corretto delle specie
        weight_tensor = torch.tensor([self.class_weights.get(species, 1.0) 
                                    for species in sorted(class_counts.keys())], dtype=torch.float32)
        self.register_buffer('alpha_weights', weight_tensor)
        
        # DEBUG: Stampa statistiche sui pesi
        weights_stats = torch.tensor(list(self.class_weights.values()))
        print(f"Pesi calcolati - Min: {weights_stats.min():.2f}, Max: {weights_stats.max():.2f}, Mean: {weights_stats.mean():.2f}")
    
    def forward(self, inputs, targets):
        device = inputs.device
        targets = targets.to(device)
        
        # BCE Loss base
        bce_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        
        # Probabilità predette (applica sigmoid per ottenere pt corretto)
        pt = torch.sigmoid(inputs)
        # Calcola pt effettivo: pt per target=1, (1-pt) per target=0
        pt = targets * pt + (1 - targets) * (1 - pt)
        
        # Componente Focal
        focal_weight = (1 - pt) ** self.gamma
        
        # Applica pesi per classe
        alpha_weights = self.alpha_weights.to(device).unsqueeze(0)
        weighted_focal_loss = alpha_weights * focal_weight * bce_loss
        
        # MIGLIORAMENTO: Media pesata con gestione migliorata degli sbilanciamenti
        # Peso alto per esempi positivi delle classi rare, peso basso per negativi
        positive_weights = targets * alpha_weights
        negative_weights = (1 - targets) * 0.05  # Peso molto basso per negativi
        
        sample_weights = positive_weights + negative_weights
        total_weight = sample_weights.sum()
        
        if total_weight > 0:
            return (weighted_focal_loss * sample_weights).sum() / total_weight
        else:
            return weighted_focal_loss.mean()

In [ ]:
class BirdCLEFSEDModel(nn.Module):
    """Modello SED IDENTICO a Bird25"""
    def __init__(self, num_classes, model_name='efficientnet_b0', pretrained=True, in_channels=3):
        super().__init__()
        
        self.num_classes = num_classes
        
        # Configura parametri come Bird25
        self.cfg = {
            'SR': config.SR,
            'hop_length': config.HOP_LENGTH,
            'n_mels': config.N_MELS,
            'f_min': config.FMIN,
            'f_max': config.FMAX,
            'n_fft': config.N_FFT,
            'normal': 80,
            'infer_duration': config.DURATION,
            'duration_train': config.DURATION
        }
        
        # Batch normalization per input (come Bird25)
        self.bn0 = nn.BatchNorm2d(config.N_MELS)
        
        # ✅ CORREZIONE: Backbone con 3 canali di input come Bird25
        self.backbone = timm.create_model(
            model_name,
            pretrained=pretrained,
            in_chans=3,  # ✅ SEMPRE 3 CANALI come Bird25
            drop_rate=0.2,
            drop_path_rate=0.2,
        )
        
        # Rimuovi gli ultimi 2 layer (come Bird25)
        layers = list(self.backbone.children())[:-2]
        self.encoder = nn.Sequential(*layers)
        
        # Per EfficientNet sempre così
        backbone_out = self.backbone.classifier.in_features
        
        # FC layer (come Bird25)
        self.fc1 = nn.Linear(backbone_out, backbone_out, bias=True)
        
        # Attention block (come Bird25)
        self.att_block = AttBlockV2(backbone_out, num_classes, activation="sigmoid")
        
        # Trasformazione Mel (come Bird25)
        self.melspec_transform = T.MelSpectrogram(
            sample_rate=self.cfg['SR'],
            hop_length=self.cfg['hop_length'],
            n_mels=self.cfg['n_mels'],
            f_min=self.cfg['f_min'],
            f_max=self.cfg['f_max'],
            n_fft=self.cfg['n_fft'],
            pad_mode="constant",
            norm="slaney",
            onesided=True,
            mel_scale="htk",
        )
        
        # Trasformazione a dB (come Bird25)
        self.db_transform = T.AmplitudeToDB(stype="power", top_db=80)
        
        self.init_weight()

    def init_weight(self):
        """Inizializza i pesi"""
        init_bn(self.bn0)
        init_layer(self.fc1)

    def extract_feature(self, x):
        """Estrae features come Bird25"""
        # x: (batch_size, channels, n_mels, time_frames)
        x = x.permute((0, 1, 3, 2))  # → (batch_size, channels, time_frames, n_mels)
        frames_num = x.shape[2]
        
        x = x.transpose(1, 3)  # → (batch_size, n_mels, time_frames, channels)
        x = self.bn0(x)
        x = x.transpose(1, 3)  # → (batch_size, channels, time_frames, n_mels)
        
        x = x.transpose(2, 3)  # → (batch_size, channels, n_mels, time_frames)
        x = self.encoder(x)
        
        # (batch_size, channels, frames)
        x = torch.mean(x, dim=2)
        
        # Channel smoothing come Bird25
        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2
        
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.transpose(1, 2)
        x = F.relu_(self.fc1(x))
        x = x.transpose(1, 2)
        x = F.dropout(x, p=0.5, training=self.training)
        
        return x, frames_num

    @torch.cuda.amp.autocast(enabled=False)
    def transform_to_spec(self, audio):
        """Trasforma audio in spettrogramma Mel"""
        audio = audio.float()
        spec = self.melspec_transform(audio)
        spec = self.db_transform(spec)
        
        if self.cfg['normal'] == 80:
            spec = (spec + 80) / 80
        elif self.cfg['normal'] == 255:
            spec = spec / 255
        
        return spec

    def forward(self, x):
        """✅ CORREZIONE: Forward pass con conversione a 3 canali"""
        # x è audio crudo (batch_size, audio_samples)
        if len(x.shape) == 2:
            x = x.unsqueeze(1)
        
        # Trasforma in spettrogramma Mel
        with torch.no_grad():
            x = self.transform_to_spec(x)  # (batch_size, n_mels, time_frames)
        
        # ✅ CORREZIONE CRITICA: Converti 1 canale in 3 canali
        if len(x.shape) == 3:  # (batch_size, n_mels, time_frames)
            x = x.unsqueeze(1)  # (batch_size, 1, n_mels, time_frames)
        
        # ✅ CONVERSIONE DA 1 A 3 CANALI (come Bird25)
        if x.shape[1] == 1:  # Se è single channel
            x = x.repeat(1, 3, 1, 1)  # Replica su 3 canali: (batch_size, 3, n_mels, time_frames)
        
        x, frames_num = self.extract_feature(x)
        (clipwise_output, norm_att, segmentwise_output) = self.att_block(x)
        
        # Calcola logit come Bird25
        logit = torch.sum(norm_att * self.att_block.cla(x), dim=2)
        
        return logit

    def infer(self, x, tta_delta=2):
        """Inference con TTA come Bird25"""
        with torch.no_grad():
            x = self.transform_to_spec(x)
        
        # ✅ CONVERSIONE DA 1 A 3 CANALI anche per inference
        if len(x.shape) == 3:
            x = x.unsqueeze(1)
        if x.shape[1] == 1:
            x = x.repeat(1, 3, 1, 1)
        
        x, _ = self.extract_feature(x)
        time_att = torch.tanh(self.att_block.att(x))
        feat_time = x.size(-1)
        
        start = 0
        end = feat_time
        pred = self.attention_infer(start, end, x, time_att)
        
        if feat_time > tta_delta * 2:
            start_minus = max(0, start - tta_delta)
            end_minus = end - tta_delta
            pred_minus = self.attention_infer(start_minus, end_minus, x, time_att)
            
            start_plus = start + tta_delta
            end_plus = min(feat_time, end + tta_delta)
            pred_plus = self.attention_infer(start_plus, end_plus, x, time_att)
            
            pred = 0.5 * pred + 0.25 * pred_minus + 0.25 * pred_plus
        
        return pred

    def attention_infer(self, start, end, x, time_att):
        """Helper per inference con attention come Bird25"""
        feat = x[:, :, start:end]
        framewise_pred = torch.sigmoid(self.att_block.cla(feat))
        framewise_pred_max = framewise_pred.max(dim=2)[0]
        return framewise_pred_max

# Controlla checkpoint esistenti
has_previous_checkpoint = False
if config.environment == 'kaggle':
    latest_checkpoint = '/kaggle/working/checkpoints/latest_checkpoint.pth'
    has_previous_checkpoint = os.path.exists(latest_checkpoint)
elif config.environment == 'colab':
    drive_checkpoint = '/content/drive/MyDrive/birdclef_checkpoints/latest_checkpoint.pth'
    has_previous_checkpoint = os.path.exists(drive_checkpoint)
else:
    local_checkpoint = os.path.join(config.OUTPUT_DIR, 'checkpoints', 'latest_checkpoint.pth')
    has_previous_checkpoint = os.path.exists(local_checkpoint)

# ⚠️ INIZIALIZZA IL NUOVO MODELLO MIGLIORATO
print("🚀 Inizializzazione del nuovo modello EfficientNet SED...")
model = BirdCLEFSEDModel(
    num_classes=config.N_CLASSES, 
    model_name='efficientnet_b0',
    pretrained=not has_previous_checkpoint,
    in_channels=3  # ✅ SEMPRE 3 canali per EfficientNet
).to(config.DEVICE)

# Aggiungi SpecAug al modello
model.spec_augmentations = spec_augmentations

print("✅ Modello SED corretto e pronto per il training!")

# Conta parametri
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"✅ Modello SED caricato su {config.DEVICE}")
print(f"📊 Parametri totali: {total_params:,}")
print(f"🎯 Parametri trainabili: {trainable_params:,}")

# Ottimizzatore per SED
def get_sed_optimizer(model, lr_backbone=5e-5, lr_head=1e-3):
    """Ottimizzatore per SED con LR differenziati"""
    backbone_params = []
    for name, param in model.named_parameters():
        if 'encoder' in name or 'backbone' in name:
            backbone_params.append(param)
    
    head_params = []
    for name, param in model.named_parameters():
        if not ('encoder' in name or 'backbone' in name):
            head_params.append(param)
    
    optimizer = optim.AdamW([
        {'params': backbone_params, 'lr': lr_backbone, 'weight_decay': 1e-4},
        {'params': head_params, 'lr': lr_head, 'weight_decay': 1e-3}
    ])
    
    return optimizer

# Calcola frequenze delle classi per la loss
primary_species_count = train_df['primary_label'].value_counts().to_dict()

# ⚠️ NUOVA LOSS WeightedFocalBCELoss con miglioramenti
criterion = WeightedFocalBCELoss(
    class_counts=primary_species_count,
    gamma=1.5,
    alpha_rare=2.5,
    alpha_common=0.6
)

# Ottimizzatore e scheduler
optimizer = get_sed_optimizer(model)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=config.EPOCHS,
    eta_min=1e-7
)

print("🎯 Setup completo! Pronto per il training...")

## 7. Addestramento e Validazione del Modello

## Funzione di Training con Supporto per Checkpoint

La funzione di training implementa:
- Caricamento automatico dei checkpoint precedenti
- Early stopping basato sulle performance di validation
- Salvataggio periodico dei checkpoint e del miglior modello
- Supporto per scheduler di learning rate (CosineAnnealingLR)
- Visualizzazione delle curve di loss

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, 
                epochs=config.EPOCHS, device=config.DEVICE, 
                model_save_path=None, model_load_path=None, patience=7,
                resume_training=True, scheduler=None):
    """
    Addestra il modello e valuta su validation set con supporto per checkpoint.
    
    Args:
        model: Modello PyTorch da addestrare
        train_loader: DataLoader per dati di training
        val_loader: DataLoader per dati di validation
        criterion: Funzione di loss
        optimizer: Ottimizzatore
        epochs: Numero di epoche di training
        device: Device per l'addestramento ('cuda' o 'cpu')
        model_save_path: Path dove salvare il modello addestrato
        model_load_path: Path da cui caricare un modello pre-addestrato
        patience: Numero di epoche senza miglioramento prima di terminare l'addestramento
        resume_training: Se True, riprende il training da un checkpoint (se disponibile)
        scheduler: Learning rate scheduler
        
    Returns:
        tuple: (train_losses, val_losses, total_training_time)
    """
    # Directory per i checkpoint in base all'ambiente
    checkpoint_dir = None
    drive_mounted = False
    
    # Configura la directory per i checkpoint a seconda dell'ambiente
    if config.environment == 'colab':
        try:
            from google.colab import drive
            # Controlla se il drive è già montato
            if not os.path.exists('/content/drive'):
                print("Montaggio di Google Drive...")
                drive.mount('/content/drive')
                print("Google Drive montato con successo.")
            
            # Crea directory per i checkpoint se non esiste
            checkpoint_dir = '/content/drive/MyDrive/birdclef_checkpoints'
            os.makedirs(checkpoint_dir, exist_ok=True)
            print(f"Directory per i checkpoint creata su Google Drive: {checkpoint_dir}")
            
            # Aggiorna il percorso di salvataggio per usare Google Drive
            if model_save_path:
                filename = os.path.basename(model_save_path)
                model_save_path = os.path.join(checkpoint_dir, filename)
                print(f"Il modello sarà salvato in: {model_save_path}")
            
            drive_mounted = True
        except ImportError:
            print("Errore: Non riesco ad accedere a Google Drive. Continuo senza persistenza.")
        except Exception as e:
            print(f"Errore durante il montaggio di Google Drive: {e}")
            print("Continuo senza persistenza su Drive.")
    elif config.environment == 'kaggle':
        # In Kaggle, usa la directory di working
        checkpoint_dir = '/kaggle/working/checkpoints'
        os.makedirs(checkpoint_dir, exist_ok=True)
        print(f"Directory per i checkpoint creata in Kaggle: {checkpoint_dir}")
    else:
        # In locale, usa la directory 'checkpoints' nell'OUTPUT_DIR
        checkpoint_dir = os.path.join(config.OUTPUT_DIR, 'checkpoints')
        os.makedirs(checkpoint_dir, exist_ok=True)
        print(f"Directory per i checkpoint creata in locale: {checkpoint_dir}")
    
    # Inizializzazione variabili
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    epochs_without_improvement = 0
    total_training_time = 0
    start_epoch = 0
    needs_training = True
    checkpoint_exists = False
    model_loaded = False
    
    # Verifica se esiste un modello pre-addestrato da caricare
    if model_load_path and os.path.exists(model_load_path):
        print(f"Modello trovato in {model_load_path}. Tentativo di caricamento...")
        try:
            checkpoint = torch.load(model_load_path, map_location=device)
            
            if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
                model.load_state_dict(checkpoint['model_state_dict'])
            else:
                model.load_state_dict(checkpoint)
                
            print("Modello caricato con successo.")
            model_loaded = True
            needs_training = False
        except Exception as e:
            print(f"Errore durante il caricamento del modello: {e}")
            print("Verrà avviato l'addestramento da zero.")
            needs_training = True
    else:
        print(f"Modello non trovato in {model_load_path}.")
    
    # Cerca un checkpoint SOLO se il caricamento del modello è fallito E resume_training è True
    if needs_training and resume_training and checkpoint_dir and not model_loaded:
        latest_checkpoint = os.path.join(checkpoint_dir, "latest_checkpoint.pth")
        if os.path.exists(latest_checkpoint):
            print(f"Trovato checkpoint in {latest_checkpoint}. Tentativo di caricamento...")
            try:
                checkpoint = torch.load(latest_checkpoint, map_location=device)
                
                # Verifica che sia un checkpoint compatibile prima di caricarlo
                if isinstance(checkpoint, dict) and 'epoch' in checkpoint:
                    try:
                        model.load_state_dict(checkpoint['model_state_dict'])
                        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                        start_epoch = checkpoint['epoch'] + 1
                        train_losses = checkpoint['train_losses']
                        val_losses = checkpoint['val_losses']
                        best_val_loss = checkpoint['best_val_loss']
                        epochs_without_improvement = checkpoint['epochs_without_improvement']
                        total_training_time = checkpoint.get('total_training_time', 0)
                        
                        # Ricrea lo scheduler con lo stato salvato se presente
                        if scheduler is not None and 'scheduler_state_dict' in checkpoint:
                            scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
                        
                        print(f"Checkpoint caricato con successo (epoca {start_epoch-1})")
                        print(f"Si riparte dall'epoca {start_epoch}/{epochs}")
                        
                        if start_epoch >= epochs:
                            needs_training = False
                        
                        checkpoint_exists = True
                    except Exception as e:
                        print(f"Il checkpoint non è compatibile con il modello attuale: {e}")
                        print("Verrà avviato l'addestramento da zero.")
            except Exception as e:
                print(f"Errore durante il caricamento del checkpoint: {e}")
                print("Si procederà con il training da zero.")
    
    model.to(device)
    
    # Esegui training solo se necessario
    if needs_training:
        start_time_total = time.time()
        model.train()
        
        # Loop di training sulle epoche (inizia da start_epoch)
        for epoch in range(start_epoch, epochs):
            epoch_start_time = time.time()
            
            # --- Fase di Training ---
            model.train()
            running_loss = 0.0
            pbar_train = tqdm(enumerate(train_loader), total=len(train_loader), 
                             desc=f"Epoca {epoch+1}/{epochs} [Train]", leave=True)
            
            for i, (inputs, labels) in pbar_train:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
                running_loss += loss.item()
                avg_loss = running_loss / (i + 1)
                pbar_train.set_postfix({'loss': f"{avg_loss:.4f}"})
            
            epoch_train_loss = running_loss / len(train_loader)
            train_losses.append(epoch_train_loss)
            
            # --- Fase di Validation ---
            model.eval()
            running_val_loss = 0.0
            pbar_val = tqdm(enumerate(val_loader), total=len(val_loader), 
                           desc=f"Epoca {epoch+1}/{epochs} [Val]", leave=True)
            
            with torch.no_grad():
                for i, (val_inputs, val_labels) in pbar_val:
                    val_inputs = val_inputs.to(device)
                    val_labels = val_labels.to(device)
                    
                    val_outputs = model(val_inputs)
                    val_loss = criterion(val_outputs, val_labels)
                    running_val_loss += val_loss.item()
                    avg_val_loss = running_val_loss / (i + 1)
                    pbar_val.set_postfix({'val_loss': f"{avg_val_loss:.4f}"})
            
            epoch_val_loss = running_val_loss / len(val_loader)
            val_losses.append(epoch_val_loss)
            
            # Aggiornamento scheduler - modificato per CosineAnnealingLR
            if scheduler is not None:
                if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                    scheduler.step(epoch_val_loss)  # Per ReduceLROnPlateau
                else:
                    scheduler.step()  # Per CosineAnnealingLR è step() senza parametri
            
            epoch_end_time = time.time()
            epoch_duration = epoch_end_time - epoch_start_time
            total_training_time += epoch_duration
            
            print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {epoch_train_loss:.4f}, "
                  f"Val Loss: {epoch_val_loss:.4f}, Duration: {epoch_duration:.2f} sec")
            
            # Salvataggio checkpoint per ogni epoca (in qualsiasi ambiente)
            if checkpoint_dir:
                checkpoint_path = os.path.join(checkpoint_dir, f"birdclef_epoch_{epoch+1}.pth")
                
                # Salva checkpoint completo con tutte le informazioni di stato
                checkpoint = {
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'train_losses': train_losses,
                    'val_losses': val_losses,
                    'best_val_loss': best_val_loss,
                    'epochs_without_improvement': epochs_without_improvement,
                    'total_training_time': total_training_time
                }
                
                # Salva anche lo stato dello scheduler se esiste
                if scheduler is not None:
                    checkpoint['scheduler_state_dict'] = scheduler.state_dict()
                
                torch.save(checkpoint, checkpoint_path)
                print(f"Checkpoint completo salvato in {checkpoint_path}")
                
                # Aggiorna anche il checkpoint più recente (sovrascrive)
                torch.save(checkpoint, os.path.join(checkpoint_dir, "latest_checkpoint.pth"))
            
            # Early stopping
            if epoch_val_loss < best_val_loss:
                best_val_loss = epoch_val_loss
                epochs_without_improvement = 0
                # Salva il miglior modello separatamente
                if model_save_path:
                    best_path = model_save_path.replace('.pth', '_best.pth')
                    
                    # Salva checkpoint completo
                    best_checkpoint = {
                        'epoch': epoch,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'train_losses': train_losses,
                        'val_losses': val_losses,
                        'best_val_loss': best_val_loss
                    }
                    
                    # Salva anche lo stato dello scheduler
                    if scheduler is not None:
                        best_checkpoint['scheduler_state_dict'] = scheduler.state_dict()
                    
                    torch.save(best_checkpoint, best_path)
                    print(f"Salvato miglior modello in {best_path}")
            else:
                epochs_without_improvement += 1
                
            if epochs_without_improvement >= patience:
                print(f"\nEarly stopping attivato! Nessun miglioramento per {patience} epoche consecutive.")
                break
        
        end_time_total = time.time()
        if checkpoint_exists:
            total_training_time += (end_time_total - start_time_total)
        else:
            total_training_time = end_time_total - start_time_total
            
        print(f"\nTraining terminato in {total_training_time/60:.2f} minuti totali")
        
        # Salva il modello finale
        if model_save_path:
            final_checkpoint = {
                'epoch': epochs-1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_losses': train_losses,
                'val_losses': val_losses,
                'best_val_loss': best_val_loss,
                'total_training_time': total_training_time
            }
            
            # Salva anche lo stato dello scheduler
            if scheduler is not None:
                final_checkpoint['scheduler_state_dict'] = scheduler.state_dict()
                
            torch.save(final_checkpoint, model_save_path)
            print(f"Modello finale salvato in {model_save_path}")
    else:
        print("Training non necessario: modello già caricato o training ripreso e completato.")
    
    # Visualizza le curve di loss
    if train_losses and val_losses:
        plt.figure(figsize=(10, 5))
        plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
        plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')
        plt.xlabel('Epoche')
        plt.ylabel('Loss')
        plt.title('Curve di Loss di Training e Validation')
        plt.legend()
        plt.grid(True)
        plt.show()
        
        # Salva il grafico
        if checkpoint_dir:
            plt_path = os.path.join(checkpoint_dir, 'loss_curves.png')
            plt.savefig(plt_path)
            print(f"Grafico delle curve di loss salvato in {plt_path}")
    
    return train_losses, val_losses, total_training_time

### Configurazione e Avvio del Training

Configuriamo e avviamo il training del modello:
- Individuazione automatica dei checkpoint precedenti
- Inizializzazione dell'ottimizzatore e dello scheduler
- Avvio del training con i parametri ottimizzati

In [ ]:
# Percorsi per caricamento/salvataggio del modello
if config.environment == 'kaggle':
    # Directory per i checkpoint in Kaggle
    os.makedirs('/kaggle/working/checkpoints', exist_ok=True)
    
    # Verifica se esiste un checkpoint precedente
    latest_checkpoint = '/kaggle/working/checkpoints/latest_checkpoint.pth'
    if os.path.exists(latest_checkpoint):
        model_load_path = latest_checkpoint
        print(f"Trovato checkpoint precedente in {latest_checkpoint}")
    else:
        # Usa un modello base precaricato se disponibile
        model_load_path = "/kaggle/input/efficientnet_data_aug/pytorch/default/1/birdclef_efficientNET_dataAug_timm_best.pth"
        
    # Imposta il percorso di salvataggio
    model_save_path = "/kaggle/working/birdclef_efficientNET_SED.pth"
    
elif config.environment == 'colab':
    # Per Colab, verifica se esiste un checkpoint su Drive
    drive_checkpoint = '/content/drive/MyDrive/birdclef_checkpoints/latest_checkpoint.pth'
    if os.path.exists(drive_checkpoint):
        model_load_path = drive_checkpoint
        print(f"Trovato checkpoint precedente su Drive: {drive_checkpoint}")
    else:
        # Altrimenti usa un modello base se disponibile
        model_load_path = os.path.join(config.MODELS_DIR, "baseline_bird_cnn_model_val.pth") if os.path.exists(os.path.join(config.MODELS_DIR, "baseline_bird_cnn_model_val.pth")) else None
    
    model_save_path = os.path.join(config.OUTPUT_DIR, f"birdclef_model_timm_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.pth")
    
else:
    # Per ambienti locali
    local_checkpoint = os.path.join(config.OUTPUT_DIR, 'checkpoints', 'latest_checkpoint.pth')
    if os.path.exists(local_checkpoint):
        model_load_path = local_checkpoint
        print(f"Trovato checkpoint precedente: {local_checkpoint}")
    else:
        model_load_path = os.path.join(config.MODELS_DIR, "baseline_bird_cnn_model_val.pth") if os.path.exists(os.path.join(config.MODELS_DIR, "baseline_bird_cnn_model_val.pth")) else None
    
    model_save_path = os.path.join(output_dirs['checkpoints'], f"birdclef_model_timm_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.pth")

# Addestra il modello con CosineAnnealingLR scheduler
train_losses, val_losses, training_time = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,  # Passa lo scheduler
    epochs=config.EPOCHS,
    device=config.DEVICE,
    model_save_path=model_save_path,
    model_load_path=model_load_path,
    resume_training=True
)

## 8. Generazione della Submission

## Generazione della Submission

Implementiamo la funzione per generare le predizioni sui file di test:
- Funzione di temporal smoothing
- Caricamento e segmentazione delle soundscape di test
- Estrazione di spettrogrammi Mel da ciascun segmento
- Generazione delle predizioni con il modello addestrato
- Creazione del file di submission nel formato richiesto dalla competizione

In [ ]:
def apply_power_to_low_ranked_cols_improved(predictions_df, top_k=30, exponent=2):
    """
    ✅ MIGLIORATA: Applica trasformazione power come Bird25
    """
    print(f"Applicando power adjustment migliorato (top_k={top_k}, exponent={exponent})...")
    
    # Salva row_id per dopo
    row_ids = predictions_df['row_id'].values
    
    # Estrai solo le colonne delle specie
    species_cols = [col for col in predictions_df.columns if col != 'row_id']
    species_data = predictions_df[species_cols].values
    
    # ✅ MIGLIORAMENTO: Identifica colonne con ranking più basso per max value
    max_values_per_col = np.max(species_data, axis=0)
    tail_col_indices = np.argsort(-max_values_per_col)[top_k:]  # Indici delle colonne "tail"
    
    # ✅ MIGLIORAMENTO: Applica trasformazione power solo dove necessario
    species_data[:, tail_col_indices] = np.power(
        np.clip(species_data[:, tail_col_indices], 0.001, 1.0),  # Evita valori 0
        exponent
    )
    
    # Ricostruisci il DataFrame
    result_df = pd.DataFrame(species_data, columns=species_cols)
    result_df.insert(0, 'row_id', row_ids)
    
    print(f"Power adjustment applicato a {len(tail_col_indices)} specie con ranking basso")
    return result_df

def enhanced_temporal_smoothing_v2(predictions_df, weights=[0.1, 0.8, 0.1]):
    """
    ✅ MIGLIORATA: Smoothing temporale più robusto
    """
    print("Applicando enhanced temporal smoothing v2...")
    
    result_df = predictions_df.copy()
    cols = [col for col in result_df.columns if col != 'row_id']
    
    # ✅ MIGLIORAMENTO: Raggruppa per file più robusto
    groups = result_df['row_id'].str.rsplit('_', n=1).str[0]
    
    for group in np.unique(groups):
        mask = groups == group
        sub_group = result_df[mask].copy()
        
        if len(sub_group) <= 1:
            continue
            
        predictions = sub_group[cols].values
        new_predictions = predictions.copy()
        
        # ✅ MIGLIORAMENTO: Smoothing adattivo
        if len(predictions) >= 3:
            # Smoothing standard per segmenti interni
            for i in range(1, len(predictions) - 1):
                new_predictions[i] = (predictions[i-1] * weights[0] + 
                                    predictions[i] * weights[1] + 
                                    predictions[i+1] * weights[2])
        
        # ✅ MIGLIORAMENTO: Gestione bordi più sofisticata
        if len(predictions) > 1:
            # Primo segmento
            new_predictions[0] = predictions[0] * 0.85 + predictions[1] * 0.15
            # Ultimo segmento
            new_predictions[-1] = predictions[-1] * 0.85 + predictions[-2] * 0.15
        
        result_df.loc[mask, cols] = new_predictions
    
    return result_df

In [ ]:
def generate_submission_sed_corrected(model, device=config.DEVICE, overlap=2.5, alpha=0.5):
    """
    ✅ CORRETTA: Genera submission usando SOLO il modello SED con pipeline corretta
    """
    model.to(device)
    model.eval()
    
    # Set seed per riproducibilità
    np.random.seed(42)
    
    # Percorso dei test soundscapes
    test_soundscape_path = config.TEST_SOUNDSCAPES_DIR
    test_soundscapes = [os.path.join(test_soundscape_path, afile) 
                        for afile in sorted(os.listdir(test_soundscape_path)) 
                        if afile.endswith('.ogg')]
    
    print(f"Elaborazione di {len(test_soundscapes)} file soundscape con overlap di {overlap}s...")
    
    # Dizionario per accumulare predizioni per ogni target timestamp
    accumulated_predictions = {}
    
    for soundscape in tqdm(test_soundscapes, desc="Elaborazione soundscapes"):
        # ✅ USA TORCHAUDIO COME NEL TRAINING
        waveform, sr = torchaudio.load(soundscape)
        
        # Converti a mono se necessario
        if waveform.shape[0] > 1:
            waveform = waveform[0:1]
        
        # Ricampiona se necessario
        if sr != config.SR:
            resampler = T.Resample(sr, config.SR)
            waveform = resampler(waveform)
        
        file_name = os.path.basename(soundscape).split('.')[0]
        audio_duration = waveform.shape[1] / config.SR
        
        # ✅ CORREZIONE: Genera timestamp ogni 5 secondi per tutto il file
        target_timestamps = list(range(5, int(audio_duration) + 1, 5))
        print(f"File {file_name}: durata {audio_duration:.1f}s, {len(target_timestamps)} timestamp da predire")
        
        # Parametri per sliding window di predizione
        window_size = int(config.SR * config.DURATION)  # 5 secondi di finestra
        step_size = int(config.SR * overlap)  # Passo di 2.5 secondi
        
        # Per ogni timestamp target (ogni 5 secondi)
        for target_time in target_timestamps:
            row_id = f"{file_name}_{target_time}"
            
            # Raccogli tutte le predizioni che coprono questo timestamp
            predictions_for_timestamp = []
            weights_for_timestamp = []
            
            # Trova tutte le finestre che coprono questo timestamp
            for start_idx in range(0, waveform.shape[1], step_size):
                segment_start_time = start_idx / config.SR
                segment_end_time = segment_start_time + config.DURATION
                
                # Se questo timestamp cade dentro questa finestra
                if segment_start_time <= target_time <= segment_end_time:
                    # Estrai segmento di 5 secondi
                    segment = waveform[:, start_idx:start_idx + window_size]
                    
                    # Se il segmento è troppo corto, padda
                    if segment.shape[1] < window_size:
                        padding = window_size - segment.shape[1]
                        segment = torch.nn.functional.pad(segment, (0, padding))
                    
                    # ✅ PASSA AUDIO CRUDO DIRETTAMENTE AL MODELLO SED
                    audio_tensor = torch.tensor(segment.squeeze().numpy(), dtype=torch.float32).unsqueeze(0).to(device)

                    with torch.no_grad():
                        # ✅ USA IL METODO INFER DEL MODELLO SED
                        output = model.infer(audio_tensor)  # Il modello gestisce internamente tutto
                        scores = output.cpu().numpy()
                    
                    # Calcola il peso in base alla posizione del timestamp nella finestra
                    relative_position = (target_time - segment_start_time) / config.DURATION
                    # Peso massimo al centro (0.5), minimo ai bordi (0, 1)
                    distance_from_center = abs(relative_position - 0.5)
                    weight = 1.0 - distance_from_center * (1.0 - alpha)
                    
                    predictions_for_timestamp.append(scores)
                    weights_for_timestamp.append(weight)
            
            # Combina le predizioni pesate per questo timestamp
            if predictions_for_timestamp:
                predictions_array = np.array(predictions_for_timestamp)
                weights_array = np.array(weights_for_timestamp)
                
                # Media pesata
                weighted_prediction = np.average(predictions_array, axis=0, weights=weights_array)
                accumulated_predictions[row_id] = weighted_prediction
            else:
                # Se nessuna finestra copre questo timestamp, usa predizioni zero
                accumulated_predictions[row_id] = np.zeros(len(all_species))
    
    # Crea il DataFrame di submission dalle predizioni accumulate
    predictions_list = []
    for row_id, scores in accumulated_predictions.items():
        predictions_list.append([row_id] + list(scores))
    
    predictions = pd.DataFrame(predictions_list, columns=['row_id'] + all_species)
    
    # Ordina per row_id per avere un output coerente
    predictions = predictions.sort_values('row_id').reset_index(drop=True)
    
    # Applica lo smoothing temporale
    print("Applicazione smoothing temporale alle predizioni...")
    predictions = enhanced_temporal_smoothing_v2(predictions)
    
    # Clip dei valori tra 0 e 1 per sicurezza
    for col in predictions.columns:
        if col != 'row_id':
            predictions[col] = predictions[col].clip(0, 1)

    # Salva la submission come CSV
    predictions.to_csv("submission.csv", index=False)
    print(f"Submission salvata con {len(predictions)} predizioni.")
    
    return predictions

In [ ]:
if config.environment == 'kaggle':
    print("\n🚀 Generazione submission SED corretta...")
    
    # ✅ USA LA FUNZIONE CORRETTA
    submission_df = generate_submission_sed_corrected(model, overlap=2.5, alpha=0.5)
    
    if submission_df is not None:
        print("\n🔄 Applicando post-processing migliorato...")
        
        # 1. Power adjustment migliorato
        submission_df = apply_power_to_low_ranked_cols_improved(
            submission_df, 
            top_k=30,      
            exponent=2     
        )
        
        # 2. Enhanced temporal smoothing v2
        submission_df = enhanced_temporal_smoothing_v2(
            submission_df,
            weights=[0.1, 0.8, 0.1]  
        )
        
        # 3. Final clipping e validazione
        species_cols = [col for col in submission_df.columns if col != 'row_id']
        submission_df[species_cols] = submission_df[species_cols].clip(0, 1)
        
        # 4. Salva submission finale
        submission_df.to_csv("submission.csv", index=False)
        print(f"✅ Submission SED corretta salvata con {len(submission_df)} predizioni")
        
        print("\n📊 Anteprima submission finale:")
        print(submission_df.head())
        
        # Statistiche di validazione
        print(f"\n📈 Statistiche submission:")
        print(f"- Numero predizioni: {len(submission_df)}")
        print(f"- Range valori: [{submission_df[species_cols].min().min():.4f}, {submission_df[species_cols].max().max():.4f}]")
        print(f"- Media predizioni per timestamp: {submission_df[species_cols].mean().mean():.4f}")
        
else:
    print("\n⏭️ Salto la generazione della submission perché non siamo su Kaggle.")